In [2]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 2.7MB 34.1MB/s 


In [3]:
from konlpy.tag import Okt
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Bag Of Words (BOW)

Bag Of Words (BOW): 단어들의 등장 횟수로 표현 (단어가방)

## BOW 만드는 과정
1. 주어진 단어에 대해 고유의 인덱스 부여
2. 단어의 등장 횟수 벡터 생성

In [3]:
from konlpy.tag import Okt
okt = Okt()

/usr/local/lib/python3.6/dist-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [19]:
token = "오늘은 금요일입니다. 내일은 토요일입니다. 다음주 화요일에는 특강이 있습니다."
# 정규표현식 사용하여 token에 저장된 문자 중에서 (.)을 제거하세요
import re
token = re.sub("\.", "", token)
token

'오늘은 금요일입니다 내일은 토요일입니다 다음주 화요일에는 특강이 있습니다'

In [20]:
token = okt.morphs(token)
token

['오늘',
 '은',
 '금요일',
 '입니다',
 '내일',
 '은',
 '토요일',
 '입니다',
 '다음주',
 '화요일',
 '에는',
 '특강',
 '이',
 '있습니다']

## BOW 만들기

In [0]:
token_indices = {}
idx = 0
token_freq = []

for i in range(len(token)):

  # 주어진 단어에 대해 고유의 인덱스 부여
  if token[i] not in token_indices:
    idx += 1
    token_indices[token[i]]=idx
  # 단어의 등장 횟수 벡터 생성 (미완)

In [22]:
token_indices

{'금요일': 3,
 '내일': 5,
 '다음주': 7,
 '에는': 9,
 '오늘': 1,
 '은': 2,
 '이': 11,
 '입니다': 4,
 '있습니다': 12,
 '토요일': 6,
 '특강': 10,
 '화요일': 8}

In [23]:
# 강사님 코딩

word2index = {}
bow = []

for voc in token:
  if voc not in word2index.keys():
    word2index[voc] = len(word2index)
    bow.insert(len(word2index)-1, 1)
  else:
    index = word2index.get(voc)
    bow[index] = bow[index] + 1

print(word2index)
print(bow)

{'오늘': 0, '은': 1, '금요일': 2, '입니다': 3, '내일': 4, '토요일': 5, '다음주': 6, '화요일': 7, '에는': 8, '특강': 9, '이': 10, '있습니다': 11}
[1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1]


## CountVectorizer

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [28]:
text = ["you know I want your love. because I love you."]
vec = CountVectorizer()    # CountVectorizer(): BOW를 만들어주는 클래스
vec.fit_transform(text)    # fit_transform함수가 주어진 텍스트를 BOW로 변환    # 각 단어별 개수를 세면서 변환
print(vec.vocabulary_)

{'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


In [30]:
text = ["you know I want your love. because I love you."]
vec = CountVectorizer()
vec.fit_transform(text).toarray()

array([[1, 1, 2, 1, 2, 1]])

In [31]:
text = ["you know I want your love. because I love you."]
vec = CountVectorizer()
print(vec.fit_transform(text).toarray())    # 각 인덱스 넘버에 해당하는 토큰이 몇 번 등장하는지 횟수
print(vec.vocabulary_)     # 각 토큰과 그 토큰의 인덱스 넘버 (인덱스 넘보는 단어 등장 순서와 일치하지는 않는다, 아마 알파벳 순서)
# 예를 들어 본 예시에서는 0번 인덱스인 "because"가 1번 등장, 1번 인덱스인 "know"가 1번 등장, 2번 인덱스인 "love"가 2번 등장, ...

[[1 1 2 1 2 1]]
{'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


In [36]:
text = ["you know I want your love. because I love you."]
# vec = CountVectorizer(stop_words=["I"])    # 불용어 사용 (사용자 정의)
vec = CountVectorizer(stop_words="english")    # 불용어 사용 (기존 불용어 사전 사용)
print(vec.fit_transform(text).toarray())
print(vec.vocabulary_)

[[1 2 1]]
{'know': 0, 'want': 2, 'love': 1}


# 문장간 유사도 조사

(1, target) 오늘 역삼에서 맛있는 돈가스를 먹었다.

(2, source) 역삼에서 먹었던 오늘의 돈가스는 맛있었다.

두 문장의 유사도를 구하면? (n-gram을 이용하여)

예: n=2라면
* "오늘", "늘 ", " 역", "역삼", "삼에", ... , "다."
* "역삼", "삼에", "에서", "서 ", " 먹", ... , "다."

(2)번 원문장에 대해 (1)번 대상문장이 어느 정도 유사한지 출력?

=> (1)번 문장을 n=2로 하여 문장 분리한 길이가 20이라 가정하고, (2)번 원문장에 대해서도 n=2로 하여 문장 분리했을 때 공통으로 존재하는 단어가 5개라면, 5/20 = 25%의 유사도를 갖는다.

In [0]:
t_text = "오늘 역삼에서 맛있는 돈가스를 먹었다."
s_text = "역삼에서 먹었던 오늘의 돈가스는 맛있었다."

In [0]:
def text_similarity(target, source, n):    # n-gram으로 나눌 때 문장 유사도
  
  #n-gram으로 나누기
  tL = []
  sL = []  
  for i in range(len(target)-(n-1)):
    tL.append(target[i:i+n])
  for i in range(len(source)-(n-1)):
    sL.append(source[i:i+n])
  
  # 유사도 구하기
  num_common = 0
  sim = 0
  for i in range(len(tL)):
    if tL[i] in sL:
      num_common += 1
  sim = num_common / len(tL)

  return sim

In [48]:
text_similarity(t_text, s_text, 2)

0.75

In [49]:
text_similarity(t_text, s_text, 3)

0.42105263157894735

# tf-idf

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
corpus = ["you know I want your love",
          "I like you",
          "what should I do"]

## DTM 생성

In [0]:
vec = CountVectorizer()

In [54]:
print(vec.fit_transform(corpus).toarray())
print(vec.vocabulary_)

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


## tf-idf 구하기

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfv = TfidfVectorizer().fit(corpus)
print("=" * 50)
print(tfidfv)
print("=" * 50)
print(tfidfv.transform(corpus).toarray())
print("=" * 50)
print(tfidfv.vocabulary_)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)
[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


# 베이시안(Bayesian) 분류기

In [2]:
from sklearn.datasets import fetch_20newsgroups

In [4]:
newsdata = fetch_20newsgroups(subset="train")
print(newsdata)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
print(newsdata.keys())
print(len(newsdata.data), len(newsdata.filenames), len(newsdata.target_names))
print(newsdata.target_names)
print(newsdata.target[0])
print(newsdata.target_names[7])

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])
11314 11314 20
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
7
rec.autos


In [13]:
print(newsdata.data[1])

From: guykuo@carson.u.washington.edu (Guy Kuo)
Subject: SI Clock Poll - Final Call
Summary: Final call for SI clock reports
Keywords: SI,acceleration,clock,upgrade
Article-I.D.: shelley.1qvfo9INNc3s
Organization: University of Washington
Lines: 11
NNTP-Posting-Host: carson.u.washington.edu

A fair number of brave souls who upgraded their SI clock oscillator have
shared their experiences for this poll. Please send a brief message detailing
your experiences with the procedure. Top speed attained, CPU rated speed,
add on cards and adapters, heat sinks, hour of usage per day, floppy disk
functionality with 800 and 1.4 m floppies are especially requested.

I will be summarizing in the next two days, so please add to the network
knowledge base if you have done the clock upgrade and haven't answered this
poll. Thanks.

Guy Kuo <guykuo@u.washington.edu>



In [14]:
print(newsdata.target[1])    # target은 데이터가 어떤 종류인지 번호로 분류    # data[1]의 target이 4번

4


In [15]:
print(newsdata.target_names[4])    # target name은 각 분류의 이름    # target 4의 target name이 comp.sys.mac.hardward

comp.sys.mac.hardware


In [17]:
testdata = fetch_20newsgroups(subset="test")
print(testdata.keys())
print(len(testdata))

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])
5


## 1. 텍스트 -> BOW 생성

In [18]:
dtmVector = CountVectorizer()
xtrainDtm = dtmVector.fit_transform(newsdata.data)
xtrainDtm.shape

(11314, 130107)

## 2. DTM -> tf-idf 행렬로 변환

In [22]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(xtrainDtm)

## 3. 나이브베이즈 분류기 모델 생성

In [24]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(tfidfv, newsdata.target)    # alpha가 라플라스 스무딩 값

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

### 만들어진 모델로 test data 돌려보고 정확도 측정하기

In [27]:
testdata = fetch_20newsgroups(subset="test", shuffle=True)

In [28]:
# BOW 생성
xtestDtm = dtmVector.transform(testdata.data)
# DTM -> tf-idf 행렬로 변환
tfidf_test = tfidf_transformer.transform(xtestDtm)

In [30]:
from sklearn.metrics import accuracy_score

predicted = model.predict(tfidf_test)
print("정확도: ", accuracy_score(testdata.target, predicted))

정확도:  0.7738980350504514
